# Introduction to crunch-shake

crunch-shake is a library aimed to help analyze plays/scripts for gender disparities. Given a script, first you have to parse it to the format specified by the library. Then you can do fun stuff like seeing what are the most common words that females or males used, run network analysis to see who are the most important characters, create a graph of plays and even run the [bechdel test](https://en.wikipedia.org/wiki/Bechdel_test).

First lets take a look at the play we will be parsing, *Romeo and Juliet* by William Shakespeare. Ever wanted to know who was the more important of the romantic duo, Romeo or Juliet? (Hint, it does not dispell any notions that we live in a patriarchy.) I've taken the play from [MIT's website](http://shakespeare.mit.edu/romeo_juliet/full.html).

In [20]:
from utils import file_to_list

romeo_juliet_raw = file_to_list("romeo_and_juliet_entire_play.html")

# Showing the beggining
for line in romeo_juliet_raw[:10]:
    print(line,end="")


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"
 "http://www.w3.org/TR/REC-html40/loose.dtd">
 <html>
 <head>
 <title>Romeo and Juliet: Entire Play
 </title>
 <meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
 <LINK rel="stylesheet" type="text/css" media="screen"
       href="/shake.css">
 </HEAD>


So obviously there's some stuff here thats not really relevant to us; lets look at some stuff in the middle of the play

In [18]:
# Showing the middle portion
for line in romeo_juliet_raw[2992:3007]:
    print(line,end="")

<A NAME=speech81><b>ROMEO</b></a>
<blockquote>
<A NAME=2.4.177>And stay, good nurse, behind the abbey wall:</A><br>
<A NAME=2.4.178>Within this hour my man shall be with thee</A><br>
<A NAME=2.4.179>And bring thee cords made like a tackled stair;</A><br>
<A NAME=2.4.180>Which to the high top-gallant of my joy</A><br>
<A NAME=2.4.181>Must be my convoy in the secret night.</A><br>
<A NAME=2.4.182>Farewell; be trusty, and I'll quit thy pains:</A><br>
<A NAME=2.4.183>Farewell; commend me to thy mistress.</A><br>
</blockquote>

<A NAME=speech82><b>Nurse</b></a>
<blockquote>
<A NAME=2.4.184>Now God in heaven bless thee! Hark you, sir.</A><br>
</blockquote>


In additions to dialogue we also have to watch out for act and scene information. 

In [23]:
for line in romeo_juliet_raw[3315:3317]:
    print(line,end="")

<H3>ACT III</h3>
<h3>SCENE I. A public place.</h3>


As well as information regarding when characters enter and exit. These stage directions can happen between dialogues, or within a dialogue (indicating a character should enter/exit while another is speaking).

In [42]:
# in between dialogue
for line in romeo_juliet_raw[1778:1782]:
    print(line,end="")

<h3>SCENE I. A lane by the wall of Capulet's orchard.</h3>
<p><blockquote>
<i>Enter ROMEO</i>
</blockquote>


In [31]:
# within a dialogue
for line in romeo_juliet_raw[3257:3273]:
    print(line,end="")

<A NAME=speech3><b>FRIAR LAURENCE</b></a>
<blockquote>
<A NAME=2.6.9>These violent delights have violent ends</A><br>
<A NAME=2.6.10>And in their triumph die, like fire and powder,</A><br>
<A NAME=2.6.11>Which as they kiss consume: the sweetest honey</A><br>
<A NAME=2.6.12>Is loathsome in his own deliciousness</A><br>
<A NAME=2.6.13>And in the taste confounds the appetite:</A><br>
<A NAME=2.6.14>Therefore love moderately; long love doth so;</A><br>
<A NAME=2.6.15>Too swift arrives as tardy as too slow.</A><br>
<p><i>Enter JULIET</i></p>
<A NAME=2.6.16>Here comes the lady: O, so light a foot</A><br>
<A NAME=2.6.17>Will ne'er wear out the everlasting flint:</A><br>
<A NAME=2.6.18>A lover may bestride the gossamer</A><br>
<A NAME=2.6.19>That idles in the wanton summer air,</A><br>
<A NAME=2.6.20>And yet not fall; so light is vanity.</A><br>
</blockquote>
